# Part I. ETL Pipeline for Pre-Processing the Files

### PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstname','gender','iteminsession','lastname','length',\
                'level','location','sessionid','song','userid'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

file = 'event_datafile_new.csv'
print("event_datafile_new.csv SUCCESSFULLY CREATED")

event_datafile_new.csv SUCCESSFULLY CREATED


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of your project. 

## The event_datafile_new.csv contains the following columns (LINE\[n\]): 
#### 0. artist \[artist\]
#### 1. firstName of user \[firstname\]
#### 2. gender of user \[gender\]
#### 3. item number in session \[iteminsession\]
#### 4. last name of user \[lastname\]
#### 5. length of the song \[length\]
#### 6. level (paid or free song) \[level\]
#### 7. location of the user \[location\]
#### 8. sessionId \[sessionid\]
#### 9. song title \[song\]
#### 10. userId \[userid\]


The image below is a screenshot of the denormalized data in <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## DATABASE CONFIGURATION

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

print("DATABASE CONNECT")

DATABASE CONNECT


#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

print("KEYSPACE CONNECT\n")

KEYSPACE CONNECT



## QUERIES OVERVIEW

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


### Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### Primary Key: Partition Key -> sessionid; Cluster Key -> iteminsession
##### - sessionid is the partition key since we are filtering the data to select rows with a specific sessionid
##### - iteminsession is a cluster key since we are filtering the data further to only display rows with a specific sessionid AND specific iteminsession

#### Presentation

In [8]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

#### Create Table 1

In [9]:
query = "CREATE TABLE IF NOT EXISTS song_session_hist"
query= query + "(sessionid int, iteminsession int, artist text, song text, length float, PRIMARY KEY (sessionid, iteminsession))"

try:
    session.execute(query)
except Exception as e:
    print(e)


In [10]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_session_hist(sessionid, iteminsession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), str(line[1]), str(line[9]), float(line[5])))

print("song_session_hist TABLE CREATED")

song_session_hist TABLE CREATED


#### Validate Query 1

In [11]:
query = "SELECT artist, song, length FROM song_session_hist WHERE sessionid = 338 AND iteminsession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

'''
index = 0
for row in rows:
    index += 1
    print ("Row %d: Artist->%s; Song->%s; length->%f" % (index, row.artist, row.song, row.length))
        # (u'Ava', u'Music Matters (Mark Knight Dub)', 495.30731201171875)
''' 

df = rows._current_rows
print(df)

print("\nQUERY 1 COMPLETE\n")

  artist                             song      length
0    Ava  Music Matters (Mark Knight Dub)  495.307312

QUERY 1 COMPLETE



### Query 2: Give me only the following: name of artist, song (sorted by iteminsession) and user (first and last name) for userid = 10, sessionid = 182

#### Primary Key: Partition Key -> userID, sessionID; Cluster Key -> iteminsession
##### - userid and sessionid are the partition keys since we are selecting the rows by a specific session no. of a particular user.
##### - iteminsession is the cluster key since we would like to sort our results by this column

#### Create Table 2

In [12]:
query = "CREATE TABLE IF NOT EXISTS user_session_hist"
query = query + "(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text, PRIMARY KEY ((userid, sessionid), iteminsession))"

try:
    session.execute(query)
except Exception as e:
    print(e)   

In [13]:
with open(file, 'r', encoding = 'utf8') as f: 
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_hist (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4])))

print("user_session_hist TABLE CREATED")

user_session_hist TABLE CREATED


#### Validate Query 2

In [14]:
query = "SELECT artist, song, firstname, lastname FROM user_session_hist WHERE userid=10 AND sessionid=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
'''
index = 0
for row in rows:
    index += 1
    print ("Row %d: Artist->%s; Song->%s; firstname->%s; lastname->%s" % (index, row.artist, row.song, row.firstname, row.lastname))
'''
df = rows._current_rows
print(df)

print("\nQUERY 2 COMPLETE\n")

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  

QUERY 2 COMPLETE



### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Primary Key: Partition Key -> song, userid; 
##### - song and userid are the partition keys since we are filtering the rows to list all users who played a particular song during any of their sessions 

#### Create Table 3

In [15]:
query = "CREATE TABLE IF NOT EXISTS song_usage_hist"
query = query + "(song text, userid int, firstname text, lastname text, PRIMARY KEY (song, userid))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
with open(file, 'r', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_usage_hist (song, userid, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (str(line[9]), int(line[10]), str(line[1]), str(line[4])))

print("song_usage_hist COMPLETE\n")

song_usage_hist COMPLETE



#### Validate Query 3

In [17]:
query = "SELECT firstname, lastname FROM song_usage_hist WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

'''
index = 0
for row in rows: 
    index += 1
    print ("Row %d: firstname->%s; lastname->%s" % (index, row.firstname, row.lastname))
'''

df = rows._current_rows
print(df)
    
print("\nQUERY 3 COMPLETE\n")

    firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson

QUERY 3 COMPLETE



### Drop Tables

In [18]:
tables = ["song_session_hist", "user_session_hist", "song_usage_hist"]
for table in tables:
    query = "DROP TABLE IF EXISTS " + table
    session.execute(query) 

print("DROP ALL TABLES")

DROP ALL TABLES


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()

print("CONNECTION CLOSE")